Gathering Pay Per View numbers for UFC and other sports events from https://www.tapology.com/

In [41]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import urllib.request

links = []

parser = 'html.parser'  
resp = urllib.request.urlopen("https://www.tapology.com/search/mma-event-figures/ppv-pay-per-view-buys-buyrate")
soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

# Getting all links, then finding page 2 results, page 3 results, etc 
for link in soup.find_all('a', href=True):
    if "/search/mma-event-figures/ppv-pay-per-view-buys-buyrate?page=" in link['href'] and "https://www.tapology.com" + link['href'] not in links:
        links.append("https://www.tapology.com" + link['href'])
        print(link['href'])

/search/mma-event-figures/ppv-pay-per-view-buys-buyrate?page=2


In [42]:
ppvs = pd.read_html("https://www.tapology.com/search/mma-event-figures/ppv-pay-per-view-buys-buyrate")[0]
for i in links:
    temp = pd.read_html(i)[0]
    ppvs = ppvs.append(temp)
    
ppvs.head()

,Event,Unnamed: 1,Name,Unnamed: 3,Date,Unnamed: 5,Buyrate
0,Mayweather vs. Pacquiao,NaN,NaN,NaN,2015.05.02,NaN,4600000
1,UFC 229,NaN,Khabib vs. McGregor,NaN,2018.10.06,NaN,2400000
2,Mayweather vs. Canelo,NaN,The One,NaN,2013.09.14,NaN,2200000
3,UFC 202,NaN,Diaz vs. McGregor 2,NaN,2016.08.20,NaN,1600000
4,Mayweather vs. Cotto,NaN,NaN,NaN,2012.05.05,NaN,1500000


In [43]:
# removing unneeded columns
for i in ppvs.columns:
    if "Unnamed" in i:
        print(i)
        ppvs.drop(i,axis = 1,inplace=True)

count = 0  
for i in ppvs["Date"]:
    ppvs["Date"].iloc[count] = i.replace(".", "-")
    count+= 1
    
ppvs.head()

Unnamed: 1
Unnamed: 3
Unnamed: 5


,Event,Name,Date,Buyrate
0,Mayweather vs. Pacquiao,NaN,2015-05-02,4600000
1,UFC 229,Khabib vs. McGregor,2018-10-06,2400000
2,Mayweather vs. Canelo,The One,2013-09-14,2200000
3,UFC 202,Diaz vs. McGregor 2,2016-08-20,1600000
4,Mayweather vs. Cotto,NaN,2012-05-05,1500000


In [47]:
import psycopg2
import requests

con = psycopg2.connect("Connection String")
print("Database opened successfully")
print()
cur = con.cursor()
cur.execute('SELECT * from "PPVS"')
tupples = cur.fetchall()
headers = [i[0] for i in cur.description]
# Dataframe created from tupples and headers
PPVS = pd.DataFrame(tupples, columns=headers)


PPVS.tail(3)

Database opened successfully



,Event,Name,Date,Buyrate
212,Strikeforce,Shamrock vs. Baroni,2007-06-22,35000
213,Pride 32,The Real Deal,2006-10-21,30000
214,Joshua vs. Klitschko,None,2017-04-29,1


In [48]:
new = pd.concat([ppvs,PPVS]).drop_duplicates(keep=False)
new.head()

,Event,Name,Date,Buyrate


In [45]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base


# Gather data for new entries

con = psycopg2.connect("Connection String")


ppvs.to_sql(
    'PPVS', 
    engine,
    index=False,
    if_exists = 'append',
    method='multi'
)